#: 모델 저장·로딩·버전 관리
- PipelineModel.save / load, 경로 전략
- 산출물: 모델 디렉토리 구조, 버전 기록표

In [1]:
import os
import sys
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.sql import SparkSession

IN_COLAB = "google.colab" in sys.modules
BASE = "/content" if IN_COLAB else os.getcwd()
CSV_PATH = os.path.join(BASE, "Social_Network_Ads.csv")
MODEL_DIR = os.path.join(BASE, "saved_models")
SEED = 42

spark = SparkSession.builder.appName("Model_SaveLoad").getOrCreate()

In [2]:
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(CSV_PATH)
indexer = StringIndexer(inputCol="Gender", outputCol="Gender_idx").setHandleInvalid("keep")
encoder = OneHotEncoder(inputCols=["Gender_idx"], outputCols=["Gender_ohe"])
assembler = VectorAssembler(inputCols=["Age", "EstimatedSalary", "Gender_ohe"], outputCol="features")
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")
lr = LogisticRegression(featuresCol="scaled_features", labelCol="Purchased")
pipeline = Pipeline(stages=[indexer, encoder, assembler, scaler, lr])

In [3]:
# 파이프라인 학습: 전처리 + 모델을 한 번에 fit
model = pipeline.fit(df)

# 모델 저장 경로 설정 (MODEL_DIR 하위에 버전별 디렉토리)
path_v1 = os.path.join(MODEL_DIR, "lr_pipeline_v1")

# 학습된 파이프라인 모델 저장 (이미 존재하면 덮어쓰기)
model.write().overwrite().save(path_v1)

# 저장 경로 확인 출력
print("Saved to", path_v1)

Saved to /content/saved_models/lr_pipeline_v1


In [4]:
from pyspark.ml.pipeline import PipelineModel

# 저장된 파이프라인 모델 불러오기 (전처리 + 모델 포함)
loaded = PipelineModel.load(path_v1)

# 원본 데이터에서 10건만 추출하여 예측 수행 (모델 로드 검증용)
predictions = loaded.transform(df.limit(10))

# 주요 피처와 예측 결과 출력
predictions.select("Age", "EstimatedSalary", "prediction").show()

+---+---------------+----------+
|Age|EstimatedSalary|prediction|
+---+---------------+----------+
| 19|          19000|       0.0|
| 35|          20000|       0.0|
| 26|          43000|       0.0|
| 27|          57000|       0.0|
| 19|          76000|       0.0|
| 27|          58000|       0.0|
| 27|          84000|       0.0|
| 32|         150000|       1.0|
| 25|          33000|       0.0|
| 35|          65000|       0.0|
+---+---------------+----------+



In [5]:
spark.stop()